In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd
import numpy as np

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
start_time = pd.Timestamp.now()

#### BASIS OPZETTEN

In [ ]:
# define network name, base dir
network_name = "tki_hooge_raam"
base_dir = Path("..\\..\\ribasim_lumping_data\\")
# directory D-HYDRO basis calculations
dhydro_basis_dir = Path(base_dir, network_name, "2_dhydro\\basis\\")
# directory D-HYDRO results calculations
dhydro_results_dir = Path(base_dir, network_name, "2_dhydro\\results\\")
# directory results
results_dir = Path(base_dir, network_name, "3_ribasim\\")
# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    crs=28992
)
# network.export_to_geopackage()

#### INLEZEN NETWERK EN SIMULATIE DATA

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_areas")
areas_file_name         = "afwateringseenheden2023_v1_HoogeRaam.shp"
areas_id_column         = "CODE"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_basis_dir, "winter"),
    simulation_name="0_01",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=True,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
# network.add_basis_network(
#     source_type='dhydro',
#     set_name='winter2', 
#     model_dir=Path(dhydro_basis_dir, "winter"),
#     simulation_name="HoogeRaamRibasim.dsproj_data",
#     dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
#     dhydro_volume_tool_force=True,
#     dhydro_volume_tool_increment=0.1
# );

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_simulation_set(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_results_dir, "winter"),
    simulation_names=["0_01", "0_02", "0_05", "0_075", "0_1", "0_2", "0_5", "0_75", "1_0", "1_5", "2_0"],
    simulation_ts=[-1],
);
# network.add_simulation_set(
#     source_type='dhydro',
#     set_name="zomer",
#     model_dir=Path(dhydro_results_dir, "winter"),
#     simulation_names=["0_01", "0_02", "0_05", "0_075", "0_1", "0_2", "0_5", "0_75", "1_0", "1_5", "2_0"],
#     simulation_ts=[-1],
# );

#### OPZETTEN SIMULATIE

In [ ]:
simulation_code = "test_samenwerkingsdag"

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=False,
    edges=False,
    structures_ids_to_include=[],
    structures_ids_to_exclude=["105BBW"],
    edge_ids_to_include=[],
    edge_ids_to_exclude=[],
);

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    "weir": "TabulatedRatingCurve",
    "uniweir": "TabulatedRatingCurve",
    "pump": "Pump",
    "culvert": "TabulatedRatingCurve",
    "edge": "ManningResistance",
    "orifice": "Outlet",
    "boundary_connection": "ManningResistance"
}
# specify translation for specific split_nodes to type of ribasim-node
split_node_id_conversion = {
}

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);

In [ ]:
# Generate Ribasim model and export to geopackage
ribasim_model = network.generate_ribasim_model_complete(
    set_name='winter',
    starttime="2020-01-01 00:00:00",
    endtime="2020-02-01 00:00:00"
)

In [ ]:
# network.plot_basin_waterlevels_based_on_node_nos(
#     set_name='winter', 
#     basins_nos=[6, 9, 10, 39, 19, 57, 58, 53, 20, 38]
# );

In [ ]:
f"Duration: {pd.Timestamp.now() - start_time}"

In [ ]:
network.ribasim_model.tabulated_rating_curve.static.groupby('node_id').get_group(2).set_index('level')[['discharge']].plot()

In [ ]:
# # export resultaten voor inspectie
# network.basins_h_df.to_csv("basins_h.csv")
# network.nodes_h_df.to_csv("nodes_h.csv")
# network.nodes_gdf[["node_no", "basin", "bedlevel", "geometry"]].to_file("nodes.geojson", driver="GeoJSON")
# network.basins_gdf[["basin", "node_id", "node_no", "name", "geometry"]].to_file("basins.geojson", driver="GeoJSON")

In [ ]:
# # nodes_h_df_1 = network.map_data.mesh1d_s1.to_dataframe().unstack().mesh1d_s1
# # old_index = nodes_h_df_1.index.copy()
# # nodes_h_df = pd.concat([nodes_h_df_1[col].sort_values().reset_index(drop=True) for col in nodes_h_df_1.columns], axis=1)
# # nodes_h_df.index = old_index

# uniweirs_q_gdf = network.his_data.weirgen_discharge.to_dataframe().unstack().T
# uniweirs_q_gdf[uniweirs_q_gdf<0.0] = 0.0
# uniweirs_q_gdf = uniweirs_q_gdf.round(3)

In [ ]:
# q = uniweirs_q_gdf.loc['weirgen_discharge'].iloc[5]['winter']

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.optimize import curve_fit

# for i, weir in uniweirs_q_gdf.loc['weirgen_discharge']['winter'].iterrows():
#     x_data = weir.index.values
#     y_data = weir.values

#     def exponential_function(x, a, b):
#         return a * np.exp(b * x)

#     params, covariance = curve_fit(exponential_function, x_data, y_data)
#     a, b = params

#     y_fit = exponential_function(x_data, a, b)

#     fig, ax = plt.subplots()
#     ax.scatter(x_data, y_data, label="Data")
#     ax.plot(x_data, y_fit, "-x", label="Exponential Fit", color='red')
#     fig.show()
#     # plt.legend()
#     # plt.xlabel('X')
#     # plt.ylabel('Y')
#     # plt.title('Exponential Regression')
#     # plt.show()

In [ ]:
# def exponential_function(x, a, b):
#     return a * np.exp(b * x)

# def apply_exponential_function(df_series):
#     x_data = df_series.index.values
#     y_data = df_series.values
#     params, covariance = curve_fit(exponential_function, x_data, y_data)
#     a, b = params
#     y_fit = exponential_function(x_data, a, b)
#     df_series.values = y_fit
#     return df_series

# uniweirs_q_gdf#.loc['winter']#.apply(lambda row: apply_exponential_function(row))